In [1]:
from llama_index.llms.llama_api import LlamaAPI

with open("../keys/llama.txt", "r") as f:
    api_key = f.read()

llm = LlamaAPI(api_key=api_key)


# Boilerplate

In [12]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)


# ZeroShot

In [31]:
from pydantic import BaseModel
from typing import List

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_mins: int

class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

    

In [32]:
from llama_index.program.openai import OpenAIPydanticProgram

prompt_template_str = """\
Extract album and songs from the text provided.
For each song, make sure to specify the title and the length_mins.
{text}
"""

llm = LlamaAPI(api_key=api_key, temperature=0.0)

program = OpenAIPydanticProgram.from_defaults(
    output_cls=Album,
    llm=llm,
    prompt_template_str=prompt_template_str,
    verbose=True,
)


ValueError: OpenAIPydanticProgram only supports OpenAI LLMs. Got: <class 'llama_index.llms.llama_api.base.LlamaAPI'>